<a href="https://colab.research.google.com/github/DManiscalco/Blackjack_RL/blob/main/Prediction_Markets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Run to get rid of invalid notebook error in GitHub
import nbformat

path = "Prediction_Markets.ipynb"
nb = nbformat.read(path, as_version=4)

# Remove bad widget metadata if it exists
if "widgets" in nb["metadata"]:
    del nb["metadata"]["widgets"]

nbformat.write(nb, path)
print("Fixed notebook saved.")

Fixed notebook saved.


In [ ]:
%%capture
!pip install faiss-cpu  # need to use faiss-gpu if using gpu
!pip install kalshi_python
!pip install rapidfuzz

In [ ]:
from datetime import datetime, timezone
from dateutil import parser
import faiss
from kalshi_python import ApiClient, Configuration, EventsApi, KalshiClient, MarketsApi, SeriesApi
import numpy as np
import pandas as pd
from rapidfuzz import fuzz
import requests
import re
from sentence_transformers import SentenceTransformer, util
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import transformers
from typing import Optional
from tqdm import tqdm

### Pull all active and not closed events in Kalshi and Polymarket and then match locally

In [ ]:
# Pull all Kalshi active markets
def fetch_markets_kalshi(status: Optional[str] = None, limit: int = 1000, cursor: Optional[str] = None):
    """
    Fetch one “page” of markets from Kalshi.
    Returns JSON response with keys: "markets" (list) and maybe "cursor".
    """
    BASE_KALSHI = "https://api.elections.kalshi.com/trade-api/v2"
    params = {"limit": limit}
    if status:
        params["status"] = status
    if cursor:
        params["cursor"] = cursor

    resp = requests.get(f"{BASE_KALSHI}/markets", params=params)
    resp.raise_for_status()
    return resp.json()

def get_all_active_markets_kalshi():
    """
    Fetch all markets that are “open” (active), via pagination, and return as a list of dicts.
    """
    all_markets = []
    cursor = None

    while True:
        resp = fetch_markets_kalshi(status="open", cursor=cursor)
        markets = resp.get("markets", [])
        all_markets.extend(markets)

        # See if there’s a next page cursor
        cursor = resp.get("cursor")
        if not cursor:
            break

    return all_markets

def markets_to_dataframe_kalshi(market_list):
    """
    Convert list of market dicts into pandas DataFrame, selecting relevant fields.
    """
    # Decide which keys you want in your df; here are some examples:
    columns = [
        "ticker",
        # "event_ticker",
        "title",
        "status",
        "open_time",
        "close_time",
    ]
    # Some markets might not have all fields; so use .get
    rows = []
    for m in market_list:
        row = {col: m.get(col) for col in columns}
        rows.append(row)

    df = pd.DataFrame(rows)
    return df

In [ ]:
# Pull all active Polymarket markets
def fetch_markets_page_poly(limit=200, offset=0):
    """
    Fetch a single page of markets from Polymarket API.
    Returns a list of market dictionaries.
    """
    BASE_POLY = "https://gamma-api.polymarket.com"
    params = {
        "limit": limit,
        "offset": offset,
        "active": True,  # only active markets
        "closed": False  # only active markets
    }
    resp = requests.get(f"{BASE_POLY}/markets", params=params)
    resp.raise_for_status()
    return resp.json()  # returns a list

def get_all_active_markets_poly(limit_per_page=200, verbose=True):
    """
    Fetch all active markets using pagination automatically.
    """
    all_markets = []
    offset = 0
    page = 0

    while True:
        markets = fetch_markets_page_poly(limit=limit_per_page, offset=offset)
        if not markets:
            break

        all_markets.extend(markets)
        page += 1

        if verbose:
            print(f"Fetched page {page}, markets so far: {len(all_markets)}")

        if len(markets) < limit_per_page:
            break  # last page reached

        offset += limit_per_page

    if verbose:
        print(f"Finished fetching all active markets: {len(all_markets)} total")

    return all_markets

def markets_to_dataframe_poly(markets):
    """
    Convert a list of market dictionaries into a pandas DataFrame.
    """
    columns = [
        "id",
        "question",
        "active",
        "closed",
        "outcomes",
        "startDate",
        "endDate"]
    return pd.DataFrame([{col: m.get(col) for col in columns} for m in markets])

In [ ]:
# Function for time string to datetime safely
def parse_time_safely(val):
    """
    Safely parse Kalshi timestamp strings, even if extra data is appended.
    Returns a UTC datetime or NaT if invalid.
    """
    if not isinstance(val, str):
        return pd.NaT
    # Take only the first part up to 'Z' (ISO 8601)
    try:
        timestamp = val.split('Z')[0] + 'Z'
        dt = parser.isoparse(timestamp)
        return dt.astimezone(timezone.utc)
    except Exception:
        return pd.NaT

In [ ]:
# Run Kalshi and then clean up the data
markets_kalshi = get_all_active_markets_kalshi()
df_kalshi = markets_to_dataframe_kalshi(markets_kalshi)
print("Number of active markets:", len(df_kalshi))

# Convert to datetime
for col in ['open_time', 'close_time']:
    df_kalshi[col] = df_kalshi[col].apply(parse_time_safely)

# UTC-aware current time
now = datetime.now(timezone.utc)

# Put in some data checks to make sure the rows are what we want
df_kalshi_filtered = df_kalshi[(df_kalshi['title'].str[:3] != 'yes')
                    & (df_kalshi['title'].str[:2] != 'no')
                    & (df_kalshi['status'] == 'active')
                    & (df_kalshi['open_time'] < now)
                    & (df_kalshi['close_time'] > now)]

df_kalshi_final = df_kalshi_filtered[['ticker', 'title']]
df_kalshi_final.head()

Number of active markets: 65506


,ticker,title
1659,KXNBATOTAL-25OCT25PHXDEN-249,Phoenix at Denver: Total Points
1660,KXNBATOTAL-25OCT25PHXDEN-246,Phoenix at Denver: Total Points
1661,KXNBATOTAL-25OCT25PHXDEN-243,Phoenix at Denver: Total Points
1662,KXNBATOTAL-25OCT25PHXDEN-240,Phoenix at Denver: Total Points
1663,KXNBATOTAL-25OCT25PHXDEN-237,Phoenix at Denver: Total Points


In [ ]:
# Run Polymarket and then clean up the data
markets_poly = get_all_active_markets_poly(limit_per_page=200, verbose=False)
df_poly = markets_to_dataframe_poly(markets_poly)
print(f"\nNumber of active markets: {len(df_poly)}")
df_poly.head()

# Convert to datetime
for col in ['startDate', 'endDate']:
    df_poly[col] = df_poly[col].apply(parse_time_safely)

# UTC-aware current time
now = datetime.now(timezone.utc)

# Put in some data checks to make sure the rows are what we want
df_poly_filtered = df_poly[(df_poly['active'] == True)
                    & (df_poly['closed'] == False)
                    & (df_poly['startDate'] < now)
                    & (df_poly['endDate'] > now)
                    & (~df_poly['outcomes'].apply(lambda x: set(x) == {"Over", "Under"}))
                    ]

df_poly_final = df_poly_filtered[['id', 'question']]
df_poly_final.head()


Number of active markets: 13455


,id,question
1,516706,Fed rate hike in 2025?
2,516710,US recession in 2025?
3,516711,Fed emergency rate cut in 2025?
4,516712,Tether insolvent in 2025?
5,516713,USDT depeg in 2025?


In [ ]:
# Find matching rows between the two dataframes
# Parameters
SIM_THRESHOLD = 0.95   # minimum similarity to count as a match
N_MATCHES = 5          # stop after N Kalshi titles with good matches

# Load model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode and normalize
emb_k = model.encode(df_kalshi_final["title"].tolist(), convert_to_numpy=True, show_progress_bar=True)
emb_p = model.encode(df_poly_final["question"].tolist(), convert_to_numpy=True, show_progress_bar=True)

emb_k = emb_k / np.linalg.norm(emb_k, axis=1, keepdims=True)
emb_p = emb_p / np.linalg.norm(emb_p, axis=1, keepdims=True)

# Build FAISS index (cosine similarity)
dim = emb_p.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(emb_p)

# Search for the single best Polymarket match per Kalshi title
similarities, indices = index.search(emb_k, 1)  # only top 1

matches = []
valid_count = 0

for i, (sim_score, idx) in enumerate(zip(similarities, indices)):
    s = float(sim_score[0])
    j = int(idx[0])

    if s < SIM_THRESHOLD:
        continue  # skip if not similar enough

    # Record the match
    matches.append({
        "kalshi_ticker": df_kalshi_final.iloc[i]["ticker"],
        "kalshi_title": df_kalshi_final.iloc[i]["title"],
        "poly_id": df_poly_final.iloc[j]["id"],
        "poly_question": df_poly_final.iloc[j]["question"],
        "similarity": s
    })

    valid_count += 1
    if valid_count >= N_MATCHES:
        break  # stop after finding N valid matches

df_matches = pd.DataFrame(matches)
print(f"✅ Found {len(df_matches)} Kalshi titles with matches above {SIM_THRESHOLD}")
df_matches.head(10)

Batches:   0%|          | 0/644 [00:00<?, ?it/s]

Batches:   0%|          | 0/396 [00:00<?, ?it/s]

✅ Found 5 Kalshi titles with matches above 0.95


,kalshi_ticker,kalshi_title,poly_id,poly_question,similarity
0,KXNHLCENTRAL-26-WPG,Will Winnipeg Jets win the NHL Central Division?,629068,Will the Winnipeg Jets win the Central Division?,0.973713
1,KXNHLCENTRAL-26-STL,Will St. Louis Blues win the NHL Central Divis...,629067,Will the St. Louis Blues win the Central Divis...,0.962277
2,KXNHLCENTRAL-26-NSH,Will Nashville Predators win the NHL Central D...,629066,Will the Nashville Predators win the Central D...,0.972187
3,KXNHLCENTRAL-26-COL,Will Colorado Avalanche win the NHL Central Di...,629063,Will the Colorado Avalanche win the Central Di...,0.965010
4,KXNHLCENTRAL-26-CHI,Will Chicago Blackhawks win the NHL Central Di...,629062,Will the Chicago Blackhawks win the Central Di...,0.976368


In [ ]:
# Extend the Polymarket ids up and down to see if there is a better match
df_deeper_matches = df_matches.copy()

# === CONFIG ===
MAX_NEARBY_SEARCH = 20
MODEL_NAME = "all-MiniLM-L6-v2"
POLY_API_URL = "https://gamma-api.polymarket.com/markets/"
VERBOSE = True

# === LOAD MODEL ===
print("Loading model...")
model = SentenceTransformer(MODEL_NAME)

# === Encode Kalshi titles with ticker ===
kalshi_texts = df_deeper_matches.apply(
    lambda r: f"{r['kalshi_ticker']} | {r['kalshi_title']}", axis=1
)
emb_kalshi = model.encode(kalshi_texts.tolist(), convert_to_numpy=True, show_progress_bar=True)
emb_kalshi = emb_kalshi / np.linalg.norm(emb_kalshi, axis=1, keepdims=True)

# === FUNCTION TO FETCH POLY MARKET BY ID ===
def fetch_polymarket_question(poly_id):
    """
    Fetch market question text from Polymarket API given an ID.
    Returns None if market not found.
    """
    try:
        url = f"{POLY_API_URL}{poly_id}"
        resp = requests.get(url, timeout=10)
        if resp.status_code == 200:
            data = resp.json()
            # Return the best available text field
            return data.get("question") or data.get("slug") or ""
        else:
            if VERBOSE:
                print(f"⚠️ ID {poly_id} returned status {resp.status_code}")
            return None
    except Exception as e:
        if VERBOSE:
            print(f"❌ Error fetching ID {poly_id}: {e}")
        return None

# === LOOP THROUGH df_deeper_matches ===
results = []

for idx, row in tqdm(df_deeper_matches.iterrows(), total=len(df_deeper_matches), desc="Processing Kalshi rows"):
    kalshi_emb = emb_kalshi[idx]
    start_poly_id = int(row["poly_id"])
    nearby_matches = []

    # --- Loop upwards ---
    current_id = start_poly_id
    steps_up = 0
    while steps_up < MAX_NEARBY_SEARCH:
        current_id += 1
        question = fetch_polymarket_question(current_id)
        if not question:
            if VERBOSE:
                print(f"↑ Upwards: ID {current_id} not found or empty. Stopping upwards search.")
            break
        emb = model.encode([f"{current_id} | {question}"], convert_to_numpy=True)
        emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)
        sim = util.cos_sim(kalshi_emb, emb)[0][0].item()
        if VERBOSE:
            print(f"↑ Upwards: ID {current_id}, similarity {sim:.4f}")
        if sim < row["similarity"]:
            if VERBOSE:
                print(f"↑ Upwards: similarity below threshold. Stopping upwards search.")
            break
        nearby_matches.append((current_id, question, sim))
        steps_up += 1

    # --- Loop downwards ---
    current_id = start_poly_id
    steps_down = 0
    while steps_down < MAX_NEARBY_SEARCH:
        current_id -= 1
        if current_id <= 0:
            if VERBOSE:
                print(f"↓ Downwards: ID {current_id} invalid. Stopping downwards search.")
            break
        question = fetch_polymarket_question(current_id)
        if not question:
            if VERBOSE:
                print(f"↓ Downwards: ID {current_id} not found or empty. Stopping downwards search.")
            break
        emb = model.encode([f"{current_id} | {question}"], convert_to_numpy=True)
        emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)
        sim = util.cos_sim(kalshi_emb, emb)[0][0].item()
        if VERBOSE:
            print(f"↓ Downwards: ID {current_id}, similarity {sim:.4f}")
        if sim < row["similarity"]:
            if VERBOSE:
                print(f"↓ Downwards: similarity below threshold. Stopping downwards search.")
            break
        nearby_matches.append((current_id, question, sim))
        steps_down += 1

    # --- Include the original Polymarket match ---
    original_question = fetch_polymarket_question(start_poly_id)
    if original_question:
        emb = model.encode([f"{start_poly_id} | {original_question}"], convert_to_numpy=True)
        emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)
        sim = util.cos_sim(kalshi_emb, emb)[0][0].item()
        nearby_matches.append((start_poly_id, original_question, sim))

    # --- Sort by similarity ---
    nearby_matches = sorted(nearby_matches, key=lambda x: x[2], reverse=True)

    # --- Take top 2 matches ---
    best_match = nearby_matches[0] if nearby_matches else (None, None, None)
    second_best_match = nearby_matches[1] if len(nearby_matches) > 1 else (None, None, None)

    results.append({
        "kalshi_ticker": row["kalshi_ticker"],
        "kalshi_title": row["kalshi_title"],
        "best_poly_id": best_match[0],
        "best_poly_question": best_match[1],
        "best_similarity": round(best_match[2], 4) if best_match[2] else None,
        "second_best_poly_id": second_best_match[0],
        "second_best_poly_question": second_best_match[1],
        "second_best_similarity": round(second_best_match[2], 4) if second_best_match[2] else None
    })

# === Final results ===
df_matches_expanded = pd.DataFrame(results)
print(f"\n✅ Completed matching with nearby Polymarket API calls")
df_matches_expanded.head(10)

Loading model...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Kalshi rows:   0%|          | 0/20 [00:00<?, ?it/s]

↑ Upwards: ID 648278, similarity 0.5257
↑ Upwards: similarity below threshold. Stopping upwards search.
↓ Downwards: ID 648276, similarity 0.4181
↓ Downwards: similarity below threshold. Stopping downwards search.


Processing Kalshi rows:   5%|▌         | 1/20 [00:00<00:11,  1.69it/s]

...

Processing Kalshi rows:  95%|█████████▌| 19/20 [00:11<00:00,  1.62it/s]

↓ Downwards: ID 648729, similarity 0.6751
↓ Downwards: similarity below threshold. Stopping downwards search.
↑ Upwards: ID 648731, similarity 0.1173
↑ Upwards: similarity below threshold. Stopping upwards search.
↓ Downwards: ID 648729, similarity 0.6423
↓ Downwards: similarity below threshold. Stopping downwards search.


Processing Kalshi rows: 100%|██████████| 20/20 [00:11<00:00,  1.71it/s]


✅ Completed matching with nearby Polymarket API calls


,kalshi_ticker,kalshi_title,best_poly_id,best_poly_question,best_similarity,second_best_poly_id,second_best_poly_question,second_best_similarity
0,KXLALIGAGAME-25NOV01RMAVCF-VCF,Real Madrid vs Valencia Winner?,648277,Real Madrid vs. Valencia: Both Teams to Score,0.6979,None,None,None
1,KXLALIGAGAME-25NOV01RMAVCF-TIE,Real Madrid vs Valencia Winner?,648277,Real Madrid vs. Valencia: Both Teams to Score,0.6999,None,None,None
2,KXLALIGAGAME-25NOV01RMAVCF-RMA,Real Madrid vs Valencia Winner?,648277,Real Madrid vs. Valencia: Both Teams to Score,0.7044,None,None,None
3,KXLALIGAGAME-25NOV01ATMSEV-TIE,Atletico vs Sevilla Winner?,648275,Atletico Madrid vs. Sevilla: Both Teams to Score,0.7224,None,None,None
4,KXLALIGAGAME-25NOV01ATMSEV-SEV,Atletico vs Sevilla Winner?,648275,Atletico Madrid vs. Sevilla: Both Teams to Score,0.7191,None,None,None
5,KXLALIGAGAME-25NOV01ATMSEV-ATM,Atletico vs Sevilla Winner?,648275,Atletico Madrid vs. Sevilla: Both Teams to Score,0.7160,None,None,None
6,KXLALIGAGAME-25NOV01VILRVC-VIL,Villarreal vs Vallecano Winner?,648274,Villarreal vs. Rayo Vallecano: Both Teams to S...,0.7086,None,None,None
7,KXLALIGAGAME-25NOV01VILRVC-TIE,Villarreal vs Vallecano Winner?,648274,Villarreal vs. Rayo Vallecano: Both Teams to S...,0.7253,None,None,None
8,KXLALIGAGAME-25NOV01VILRVC-RVC,Villarreal vs Vallecano Winner?,648274,Villarreal vs. Rayo Vallecano: Both Teams to S...,0.7090,None,None,None
9,KXNFLRSHYDS-25OCT26CLENE-NETHENDERSON32-40,TreVeyon Henderson records 40+ rushing yards,544501,Will TreVeyon Henderson record the most rushin...,0.7388,None,None,None
